<a href="https://colab.research.google.com/github/KarkiAnuj17/Automatic-text-summarization/blob/main/Automatic_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/dataset.zip", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

In [3]:
import cudf
import pandas as pd
import nltk
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet


In [4]:
path="/content/dataset/dataset/test.csv"
df=pd.read_csv(path)
print(df.columns)
print(df.shape)

Index(['id', 'article', 'highlights'], dtype='object')
(11490, 3)


In [5]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [44]:
def get_wordnet_pos(treebank_tag):
    # Map POS tag to WordNet POS tag
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text):
    # Split text into sentences
    sentences = sent_tokenize(text)

    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Get English stopwords
    stop_words = set(stopwords.words('english'))

    # Preprocess each sentence
    processed_sentences = []
    for sentence in sentences:
        # Tokenize words in the sentence
        words = word_tokenize(sentence)

        # Get POS tags
        pos_tags = pos_tag(words)

        # Lemmatize and remove stopwords
        processed_words = []
        for word, pos in pos_tags:
            if word.lower() not in stop_words and word.isalnum():
                lemmatized_word = lemmatizer.lemmatize(word.lower(), get_wordnet_pos(pos))
                processed_words.append(lemmatized_word)

        # Append processed sentence
        processed_sentences.append(processed_words)

    return processed_sentences

def preprocess_csv(file_path, text_column):
    # Read the CSV file into a cuDF DataFrame
    df = cudf.read_csv(file_path)

    # Convert text column to a pandas Series for processing with NLTK
    text_series = df[text_column].to_pandas()

    # Apply preprocessing to the text column with progress bar
    processed_text = text_series.progress_apply(preprocess_text)

    # Convert the processed text back to a cuDF DataFrame
    df['processed_text'] = cudf.from_pandas(processed_text)

    return df

# Enable the tqdm progress_apply
tqdm.pandas()

file_path = '/content/dataset/dataset/test.csv'
text_column = 'article'
processed_df = preprocess_csv(file_path, text_column)

# Display the processed DataFrame
print(processed_df.head())


100%|██████████| 11490/11490 [07:42<00:00, 24.82it/s]


                                         id  \
0  92c514c913c0bdfe25341af9fd72b29db544099b   
1  2003841c7dc0e7c5b1a248f9cd536d727f27a45a   
2  91b7d2311527f5c2b63a65ca98d21d9c92485149   
3  caabf9cbdf96eb1410295a673e953d304391bfbb   
4  3da746a7d9afcaa659088c8366ef6347fe6b53ea   

                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                          highlights  \
0  Experts question if  packed out planes are put...   
1  Drunk teenage boy climbed into lion enclosure ...   
2  Nottingham Forest are close to extending Dougi...   
3  Fiorentina goalkeeper Neto has been linked wit...   
4  Tell-all interview with the reality TV star, 6...   

                                      proce

In [45]:
import cupy as cp
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

# Cosine Similarity Function
def cosine_similarity_matrix(matrix):
    matrix = cp.array(matrix)
    dot_product = cp.dot(matrix, matrix.T)  # Compute dot product
    norm = cp.linalg.norm(matrix, axis=1)  # Compute norm (magnitude) of each vector
    similarity_matrix = dot_product / (cp.outer(norm, norm) + 1e-10)  # Cosine similarity
    return similarity_matrix

# Vectorization Function using TF-IDF
def vectorize_sentences_with_tfidf(sentences):
    vectorizer = TfidfVectorizer(stop_words='english')  # Optional: Remove stopwords
    term_doc_matrix_cpu = vectorizer.fit_transform(sentences)  # Compute the TF-IDF matrix

    # Convert the sparse matrix to a dense NumPy array
    term_doc_matrix_cpu = term_doc_matrix_cpu.toarray()

    # Convert to CuPy array for GPU acceleration
    term_doc_matrix = cp.array(term_doc_matrix_cpu)

    return term_doc_matrix

# Function to Compute Similarity for Articles
def compute_similarity_for_articles(df, text_column='processed_text'):
    text_series = df[text_column].to_pandas()  # Get text data from DataFrame
    similarity_matrices = []

    # Iterate over each article's processed text
    for index, processed_text in tqdm(text_series.items(), total=len(text_series), desc="Processing Articles"):

        # Ensure processed_text is not empty and is a list of sentences
        if processed_text and isinstance(processed_text, list):
            processed_sentences = [' '.join(sentence) for sentence in processed_text]  # Join words into sentences
        else:
            processed_sentences = []

        # Skip if the processed_sentences list is empty
        if not processed_sentences:
            similarity_matrices.append(None)
            continue

        # Vectorize sentences using TF-IDF approach
        term_doc_matrix = vectorize_sentences_with_tfidf(processed_sentences)

        # Compute the cosine similarity matrix
        similarity_matrix = cosine_similarity_matrix(term_doc_matrix)

        # Convert to CPU (NumPy) array for easier handling if necessary
        similarity_matrix_cpu = cp.asnumpy(similarity_matrix)

        # Store the similarity matrix for the article
        similarity_matrices.append(similarity_matrix_cpu)

    return similarity_matrices

# Enable tqdm for progress bar in pandas apply
tqdm.pandas()

# Assuming you have a DataFrame `processed_df` with a 'processed_text' column
final_similarity_matrices = compute_similarity_for_articles(processed_df, text_column='processed_text')

# Example output for the first article
print("Final Cosine Similarity Matrix for the First Article:\n", final_similarity_matrices[0])


Processing Articles: 100%|██████████| 11490/11490 [00:41<00:00, 279.57it/s]


Final Cosine Similarity Matrix for the First Article:
 [[1.         0.03550455 0.         0.04097448 0.02367313 0.
  0.         0.07210004 0.04264119 0.0403271  0.03092843 0.07493094
  0.11954024 0.         0.02426954 0.06009226]
 [0.03550455 1.         0.         0.03528285 0.02038477 0.
  0.08077285 0.02888779 0.03671804 0.         0.08621036 0.03002202
  0.         0.         0.         0.        ]
 [0.         0.         1.         0.53670032 0.0884582  0.12443423
  0.         0.04675813 0.         0.         0.         0.
  0.         0.         0.02943534 0.        ]
 [0.04097448 0.03528285 0.53670032 1.         0.0548335  0.04404127
  0.         0.07770597 0.04237492 0.         0.0307353  0.0346473
  0.         0.         0.02793048 0.        ]
 [0.02367313 0.02038477 0.0884582  0.0548335  1.         0.21782932
  0.         0.04489486 0.07925059 0.         0.0574819  0.06479821
  0.         0.         0.01613692 0.        ]
 [0.         0.         0.12443423 0.04404127 0.2178293

In [69]:
import cupy as cp
from tqdm import tqdm

# TextRank Function
def textrank(sentences, similarity_matrix, damping_factor=0.85, max_iter=100, tol=1e-6):
    """
    Perform TextRank algorithm for ranking sentences.

    Parameters:
        sentences (list): List of sentences.
        similarity_matrix (cupy.ndarray): Precomputed similarity matrix.
        damping_factor (float): Damping factor for TextRank algorithm.
        max_iter (int): Maximum number of iterations for convergence.
        tol (float): Tolerance for convergence.

    Returns:
        list: List of tuples (sentence, score) sorted by score in descending order.
    """
    n_sentences = len(sentences)
    scores = cp.ones(n_sentences) / n_sentences  # Initialize scores uniformly

    for _ in range(max_iter):
        new_scores = cp.ones(n_sentences) * (1 - damping_factor) / n_sentences
        new_scores += damping_factor * cp.dot(similarity_matrix.T, scores)

        if cp.linalg.norm(new_scores - scores) < tol:  # Check for convergence
            break
        scores = new_scores

    scores = cp.asnumpy(scores)  # Convert to NumPy for compatibility
    ranked_sentences = sorted(zip(sentences, scores), key=lambda x: x[1], reverse=True)
    return ranked_sentences

# Function to Compute TextRank for Processed Data
def compute_textrank_for_articles(processed_df, similarity_matrices, text_column='processed_text'):
    """
    Compute TextRank scores for each article in the processed DataFrame.

    Parameters:
        processed_df (cudf.DataFrame): DataFrame containing preprocessed text data.
        similarity_matrices (list): List of similarity matrices for each article.
        text_column (str): Column name containing processed sentences.

    Returns:
        list: List of ranked sentences with scores for each article.
    """
    text_series = processed_df[text_column].to_pandas()  # Convert to pandas for processing
    all_ranked_sentences = []

    for idx, (processed_text, similarity_matrix) in tqdm(
        enumerate(zip(text_series, similarity_matrices)),
        total=len(text_series),
        desc="Computing TextRank for Articles"
    ):
        # Ensure processed_text and similarity_matrix are valid
        if not processed_text or similarity_matrix is None:
            all_ranked_sentences.append(None)
            continue

        # Prepare sentences and ensure compatibility
        sentences = [' '.join(sentence) for sentence in processed_text]
        similarity_matrix = cp.array(similarity_matrix)  # Move similarity matrix to GPU

        # Compute TextRank
        ranked_sentences = textrank(sentences, similarity_matrix)
        all_ranked_sentences.append(ranked_sentences)

    return all_ranked_sentences

# Assuming you have `processed_df` and `final_similarity_matrices` already available
ranked_sentences_per_article = compute_textrank_for_articles(processed_df, final_similarity_matrices)

# Output example for the first article
print("Ranked Sentences for the First Article:")
for sentence, score in ranked_sentences_per_article[0]:
    print(f"Score: {score:.4f} - Sentence: {sentence}")


Computing TextRank for Articles: 100%|██████████| 11490/11490 [18:57<00:00, 10.10it/s]


Ranked Sentences for the First Article:
Score: 2087802405223129178469074353520640.0000 - Sentence: test conduct faa use plane 31 inch pitch standard airline decrease
Score: 1936359890403713434337974903898112.0000 - Sentence: many economy seat united airline 30 inch room airline offer little 28 inch
Score: 1907197179605943111203070238785536.0000 - Sentence: united airline 30 inch space gulf air economy seat 29 32 inch air asia offer 29 inch spirit airline offer 28 inch
Score: 1893518571944218732339545421905920.0000 - Sentence: test conduct use plane 31 inch row seat standard airline decrease report detroit news
Score: 1815780829574358186266313913008128.0000 - Sentence: british airway seat pitch 31 inch easyjet 29 inch thomson short haul seat pitch 28 inch virgin atlantic
Score: 1765594298684167494298801029513216.0000 - Sentence: airline stick pitch 31 inch fall
Score: 813647559314011728417479460913152.0000 - Sentence: distance two seat one point seat point seat behind know pitch
Score: 

In [74]:
import cupy as cp
from tqdm import tqdm

def initialize_centroids(embeddings, n_clusters):
    """
    Randomly initialize centroids for K-means clustering.

    Parameters:
        embeddings (cupy.ndarray): The sentence embeddings or similarity matrix.
        n_clusters (int): The number of clusters to form.

    Returns:
        cupy.ndarray: Randomly initialized centroids.
    """
    n_samples = embeddings.shape[0]

    # Ensure n_clusters does not exceed the number of samples
    if n_clusters > n_samples:
        n_clusters = n_samples

    random_indices = cp.random.choice(n_samples, n_clusters, replace=False)
    centroids = embeddings[random_indices]
    return centroids

def assign_clusters(embeddings, centroids):
    """
    Assign each sentence to the nearest centroid.

    Parameters:
        embeddings (cupy.ndarray): The sentence embeddings or similarity matrix.
        centroids (cupy.ndarray): The centroids for the clusters.

    Returns:
        cupy.ndarray: Cluster labels for each sentence.
    """
    distances = cp.linalg.norm(embeddings[:, None] - centroids, axis=2)  # Compute pairwise distances
    cluster_labels = cp.argmin(distances, axis=1)  # Assign the nearest centroid
    return cluster_labels

def update_centroids(embeddings, cluster_labels, n_clusters):
    """
    Update centroids by computing the mean of the sentences in each cluster.

    Parameters:
        embeddings (cupy.ndarray): The sentence embeddings or similarity matrix.
        cluster_labels (cupy.ndarray): The current cluster labels for each sentence.
        n_clusters (int): The number of clusters.

    Returns:
        cupy.ndarray: Updated centroids.
    """
    new_centroids = cp.zeros((n_clusters, embeddings.shape[1]))
    for i in range(n_clusters):
        cluster_points = embeddings[cluster_labels == i]
        if len(cluster_points) > 0:
            new_centroids[i] = cp.mean(cluster_points, axis=0)
        else:
            # If the cluster has no points, reinitialize the centroid randomly
            random_idx = cp.random.choice(embeddings.shape[0], size=1)  # Specify size=1
            new_centroids[i] = embeddings[random_idx]
    return new_centroids

def kmeans_clustering(embeddings, n_clusters=3, max_iters=100, tol=1e-6):
    """
    Perform K-means clustering on sentence embeddings.

    Parameters:
        embeddings (cupy.ndarray): The sentence embeddings or similarity matrix.
        n_clusters (int): The number of clusters to form.
        max_iters (int): Maximum number of iterations to run the K-means algorithm.
        tol (float): Tolerance for convergence (when centroids stop changing).

    Returns:
        tuple: A tuple containing the final cluster labels and centroids.
    """
    centroids = initialize_centroids(embeddings, n_clusters)

    for _ in range(max_iters):
        cluster_labels = assign_clusters(embeddings, centroids)
        new_centroids = update_centroids(embeddings, cluster_labels, n_clusters)

        # Check for convergence (ensure shapes match before comparison)
        if new_centroids.shape == centroids.shape and cp.allclose(centroids, new_centroids, atol=tol, rtol=0):
            break

        centroids = new_centroids

    return cluster_labels, centroids

# Function to compute K-means clustering on the ranked sentences for each article
def compute_kmeans_for_articles(processed_df, similarity_matrices, n_clusters=3, text_column='processed_text'):
    """
    Perform K-means clustering on the ranked sentences of each article.

    Parameters:
        processed_df (cudf.DataFrame): DataFrame containing preprocessed text data.
        similarity_matrices (list): List of similarity matrices for each article.
        n_clusters (int): Number of clusters for K-means.
        text_column (str): Column name containing processed sentences.

    Returns:
        list: List of cluster labels for each article's ranked sentences.
    """
    text_series = processed_df[text_column].to_pandas()  # Convert to pandas for processing
    all_cluster_labels = []

    for idx, (processed_text, similarity_matrix) in tqdm(
        enumerate(zip(text_series, similarity_matrices)),
        total=len(text_series),
        desc="Computing K-means for Articles"
    ):
        # Ensure processed_text and similarity_matrix are valid
        if not processed_text or similarity_matrix is None:
            all_cluster_labels.append(None)
            continue

        # Prepare sentences and ensure compatibility
        sentences = [' '.join(sentence) for sentence in processed_text]
        similarity_matrix = cp.array(similarity_matrix)  # Move similarity matrix to GPU

        # Perform K-means clustering on the similarity matrix (treated as sentence embeddings)
        cluster_labels, _ = kmeans_clustering(similarity_matrix, n_clusters=n_clusters)
        all_cluster_labels.append(cluster_labels)

    return all_cluster_labels

# Assuming you have `processed_df` and `final_similarity_matrices` already available
cluster_labels_per_article = compute_kmeans_for_articles(processed_df, final_similarity_matrices, n_clusters=3)

# Output example for the first article
print("Cluster Labels for the First Article:")
for idx, label in enumerate(cluster_labels_per_article[0]):
    print(f"Sentence {idx} is in Cluster {label}")


Computing K-means for Articles: 100%|██████████| 11490/11490 [02:04<00:00, 92.19it/s] 


Cluster Labels for the First Article:
Sentence 0 is in Cluster 2
Sentence 1 is in Cluster 2
Sentence 2 is in Cluster 1
Sentence 3 is in Cluster 0
Sentence 4 is in Cluster 2
Sentence 5 is in Cluster 2
Sentence 6 is in Cluster 2
Sentence 7 is in Cluster 2
Sentence 8 is in Cluster 2
Sentence 9 is in Cluster 2
Sentence 10 is in Cluster 2
Sentence 11 is in Cluster 2
Sentence 12 is in Cluster 2
Sentence 13 is in Cluster 2
Sentence 14 is in Cluster 2
Sentence 15 is in Cluster 2


In [98]:
import nltk
import cupy as cp
import pandas as pd
from tqdm import tqdm
import cudf
from collections import defaultdict

# Function to Compute TextRank Scores Using Precomputed Similarity Matrices
def compute_text_rank_from_similarity(similarity_matrix, damping_factor=0.85, max_iterations=100, tol=1e-6):
    n = similarity_matrix.shape[0]
    scores = cp.ones(n) / n  # Initialize scores uniformly
    transition_matrix = similarity_matrix / (similarity_matrix.sum(axis=1, keepdims=True) + 1e-10)  # Normalize rows

    for _ in range(max_iterations):
        new_scores = (1 - damping_factor) / n + damping_factor * cp.dot(transition_matrix.T, scores)
        if cp.linalg.norm(new_scores - scores, ord=1) < tol:
            break
        scores = new_scores

    return cp.asnumpy(scores)  # Convert to NumPy for easy handling


# Function to Extract Summary Using Clustering and TextRank
def get_summary_by_cluster(processed_text, original_sentences, scores, cluster_labels, summary_length=None):
    clusters = defaultdict(list)

    # Ensure cluster labels are integers (hashable)
    cluster_labels = [int(label) for label in cluster_labels]

    # Group sentences by clusters
    for idx, score in enumerate(scores):
        cluster = cluster_labels[idx]
        clusters[cluster].append((idx, score))  # Store (sentence index, score) in the cluster

    all_sentences_with_scores = []

    # For each cluster, sort the sentences by score
    for cluster, sentence_scores in clusters.items():
        sentence_scores_sorted = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
        all_sentences_with_scores.extend(sentence_scores_sorted)  # Add all sentences in sorted order

    # Sort all sentences by score (across all clusters)
    all_sentences_with_scores_sorted = sorted(all_sentences_with_scores, key=lambda x: x[1], reverse=True)

    # Extract the top sentences based on desired summary length (if given)
    if summary_length:
        summary_sentences = sorted(all_sentences_with_scores_sorted[:summary_length], key=lambda x: x[0])
    else:
        # Default strategy: take a proportion of the sentences (e.g., top 30%)
        summary_sentences = sorted(all_sentences_with_scores_sorted[:int(len(all_sentences_with_scores_sorted) * 0.3)], key=lambda x: x[0])

    # Retrieve the original sentences in chronological order
    summary = [original_sentences[idx] for idx, _ in summary_sentences]

    return summary


# Main Function to Rank Sentences Using Original Text and Precomputed Similarity Matrices with Clustering
def generate_summary_with_clustering(df, similarity_matrices, cluster_labels_per_article, text_column='processed_text', original_text_column='original_text', summary_length=None):
    # First, convert the DataFrame to Pandas for sentence tokenization
    df_cpu = df.to_pandas()

    # Apply sentence tokenization to 'article' column (on CPU)
    df_cpu['original_text'] = df_cpu['article'].apply(
        lambda x: nltk.sent_tokenize(x)  # Split full article into sentences
    )

    # Convert back to cuDF for further processing (optional)
    df_gpu = cudf.from_pandas(df_cpu)

    text_series = df_gpu[text_column].to_pandas()  # Extract processed text
    original_text_series = df_gpu[original_text_column].to_pandas()  # Extract original text
    all_summaries = []

    for index, (processed_text, original_sentences, similarity_matrix, cluster_labels) in tqdm(
        enumerate(zip(text_series, original_text_series, similarity_matrices, cluster_labels_per_article)),
        total=len(similarity_matrices),
        desc="Generating Summaries"
    ):
        # Compute TextRank scores from the precomputed similarity matrix
        text_rank_scores = compute_text_rank_from_similarity(cp.array(similarity_matrix))

        # Get the summary grouped by clusters
        summary = get_summary_by_cluster(
            processed_text, original_sentences, text_rank_scores, cluster_labels, summary_length=summary_length
        )

        # Store the summary
        all_summaries.append(summary)

    return all_summaries


# Example Usage (ensure that `processed_df`, `final_similarity_matrices`, and `cluster_labels_per_article` are defined):
generated_summaries = generate_summary_with_clustering(
    processed_df, final_similarity_matrices, cluster_labels_per_article, text_column='processed_text', original_text_column='original_text', summary_length=10  # Adjust length as needed
)

# Output example for the first article (Chronological Order)
print("Summary for the First Article (Chronological Order):")
for sentence in generated_summaries[0]:
    print(sentence)


Generating Summaries: 100%|██████████| 11490/11490 [05:09<00:00, 37.08it/s]


Summary for the First Article (Chronological Order):
They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger.
More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger?
This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.
Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decreased .
Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches .
Cynthia Corbertt, a human factors researcher with the Federal Aviation Administration, that it conducts tests on how quickly passengers can leave a plane.
But these tests are conducted using planes with 31 inches between each row of seats, a sta

In [94]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!sudo apt-get update
!sudo apt-get install -y locales
!sudo locale-gen en_US.UTF-8
!sudo update-locale LANG=en_US.UTF-8
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
!pip install rouge-score

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,185 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,738 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/